# Generate snowflake schema and insert data into Netezza
> 1. at first I will generate data on equity transactions using faker, random and list of dummy of companies / industries  
> 2. only 4 stocks correspond to real companies ('BAC', 'IBM', 'AMZN', 'HD') as they are used by the agent  
> 3. then I will create corresponding Netezza tables, save files as csvs and insert data from local files to Netezza

![200](attachment:image.png)

In [1]:
import os
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

from dotenv import load_dotenv
import nzpy

fake = Faker('en_US')
random.seed(42)
load_dotenv('.env_load')

True

## Settings

In [2]:
# to get number of accounts
accounts_df = pd.read_csv('../Data-Preparation/gen_data/accounts.csv')
accounts_df.describe()

,account_id,customer_id
count,246301.000000,246301.000000
mean,123151.000000,61469.608175
std,71101.118663,35515.129664
min,1.000000,1.000000
25%,61576.000000,30718.000000
50%,123151.000000,61498.000000
75%,184726.000000,92187.000000
max,246301.000000,123000.000000


In [3]:
# Date range for time dimension
start_date = datetime(2019, 1, 1)
end_date = datetime(2025, 3, 31)

NUM_FCOMPANIES = 100
NUM_TRANSACTIONS = 1200000  # Number of transactions
NUM_ACCOUNTS = len(accounts_df) # Number of trading accounts

In [4]:
def df_overview(df):
    display(df.head())
    display(df.info())
    display(df.describe())

### Expected directories for temp files and logs for Netezza

In [5]:
dir_logs = './logs'
dir_nz = './tmp_nz'
# Create the directory if it doesn't exist
os.makedirs(dir_logs, exist_ok=True)
os.makedirs(dir_nz, exist_ok=True)

## Data Generation

### Stocks dimension

In [6]:
def generate_ticker(name):
    """Create a stock ticker symbol from the company name."""
    words = name.split()
    if len(words) == 1:
        return name[:4].upper()
    return (words[0][:2] + words[-1][:2]).upper()

In [7]:
# Real companies
REAL_STOCKS = [
    ('BAC', 'Bank of America', 'Financials', 'Banking'),
    ('IBM', 'IBM', 'Technology', 'IT Services'),
    ('AMZN', 'Amazon', 'Consumer Discretionary', 'E-Commerce'),
    ('HD', 'Home Depot', 'Consumer Discretionary', 'Home Improvement')
]

SECTORS_IND = list(zip(['Technology', 'Technology', 'Energy', 'Technology', 'Industrial',
         'Healthcare', 'Consumer Discretionary', 'Financials', 'Technology', 'Consumer Discretionary',
         'Technology', 'Energy', 'AI & Machine Learning', 'Healthcare',
         'Technology', 'Healthcare', 'Financials', 'Consumer Discretionary',
         'Technology', 'Cloud Computing', 'Energy', 'Aerospace', 'AI & Robotics',
         'Cybersecurity', 'Neuroscience', 'Biotechnology', 'Computing Hardware',
         'Augmented Reality', 'Cybersecurity', 'Digital Media', 'Aviation',
         'AI Research', 'Renewable Energy', 'Blockchain', 'Telecommunications',
         'Data Analytics', 'Automotive', 'Digital Services', 'Aerospace Engineering',
         'Automation'],
        ['AI Solutions', 'Quantum Computing', 'Renewable Energy', 'Cybersecurity',
         'Aerospace Engineering', 'Biotechnology', 'Electric Vehicles', 'Investment Banking',
         'Digital Advertising', 'E-Commerce', 'Robotics', 'Solar Energy',
         'Deep Learning Systems', 'Genomics', 'Satellite Communications',
         'Medical Devices', 'Wealth Management', 'Automobile Manufacturing',
         'Supercomputers', 'Cloud Services', 'Green Energy', 'Space Exploration',
         'Machine Learning', 'Network Security', 'Brain-Computer Interfaces',
         'DNA Editing', 'High-Performance Computing', 'Mixed Reality',
         'Advanced Cybersecurity', 'Streaming Media', 'Commercial Aviation',
         'AI Research', 'Solar Technology', 'Cryptocurrency', '5G Networks',
         'Big Data Analytics', 'Autonomous Vehicles', 'E-Commerce Platforms',
         'Interplanetary Travel', 'Industrial Automation']))

# Generate 40 fake companies dynamically
FAKE_COMPANY_NAMES = [fake.company() for _ in range(NUM_FCOMPANIES)]
SECTOR, INDUSTRY = zip(*[random.choice(SECTORS_IND) for _ in range(NUM_FCOMPANIES)])



In [8]:
FAKE_STOCKS = [
    (generate_ticker(name), name, sector, industry)
    for name, sector, industry in zip(
        FAKE_COMPANY_NAMES, SECTOR, INDUSTRY

    )
]

In [9]:
# Combine real and fake stocks
STOCKS = REAL_STOCKS + FAKE_STOCKS

# Generate dim_stock table
dim_stock = pd.DataFrame({
    'stock_id': range(1, len(STOCKS) + 1),
    'stock_symbol': [s[0] for s in STOCKS],
    'stock_name': [s[1] for s in STOCKS],
    'sector': [s[2] for s in STOCKS],
    'industry': [s[3] for s in STOCKS],
    'market_cap': [random.randint(100, 500) * 1e9 for _ in STOCKS]  # Market cap in billions
})

In [10]:
df_overview(dim_stock)

,stock_id,stock_symbol,stock_name,sector,industry,market_cap
0,1,BAC,Bank of America,Financials,Banking,2.260000e+11
1,2,IBM,IBM,Technology,IT Services,4.810000e+11
2,3,AMZN,Amazon,Consumer Discretionary,E-Commerce,3.870000e+11
3,4,HD,Home Depot,Consumer Discretionary,Home Improvement,3.750000e+11
4,5,SMIT,Smith-Watson,Financials,Investment Banking,2.340000e+11


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   stock_id      104 non-null    int64  
 1   stock_symbol  104 non-null    object 
 2   stock_name    104 non-null    object 
 3   sector        104 non-null    object 
 4   industry      104 non-null    object 
 5   market_cap    104 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 5.0+ KB


None

,stock_id,market_cap
count,104.000000,1.040000e+02
mean,52.500000,3.105000e+11
std,30.166206,1.202521e+11
min,1.000000,1.000000e+11
25%,26.750000,1.985000e+11
50%,52.500000,3.270000e+11
75%,78.250000,4.052500e+11
max,104.000000,4.980000e+11


### Date dimension

In [11]:
date_range = pd.date_range(start=start_date, end=end_date)
dim_date = pd.DataFrame({
    'date_id': range(1, len(date_range) + 1),
    'transaction_date': date_range,
    'year': date_range.year,
    'quarter': date_range.quarter,
    'month': date_range.month,
    'week': date_range.isocalendar().week,
    'day_of_week': date_range.dayofweek,
    'is_weekend': date_range.dayofweek >= 5
})

In [12]:
dim_date['is_weekend'] = dim_date['is_weekend'].map({True: 1, False: 0})

In [13]:
df_overview(dim_date)

,date_id,transaction_date,year,quarter,month,week,day_of_week,is_weekend
2019-01-01,1,2019-01-01,2019,1,1,1,1,0
2019-01-02,2,2019-01-02,2019,1,1,1,2,0
2019-01-03,3,2019-01-03,2019,1,1,1,3,0
2019-01-04,4,2019-01-04,2019,1,1,1,4,0
2019-01-05,5,2019-01-05,2019,1,1,1,5,1


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2282 entries, 2019-01-01 to 2025-03-31
Freq: D
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date_id           2282 non-null   int64         
 1   transaction_date  2282 non-null   datetime64[ns]
 2   year              2282 non-null   int32         
 3   quarter           2282 non-null   int32         
 4   month             2282 non-null   int32         
 5   week              2282 non-null   UInt32        
 6   day_of_week       2282 non-null   int32         
 7   is_weekend        2282 non-null   int64         
dtypes: UInt32(1), datetime64[ns](1), int32(4), int64(2)
memory usage: 118.1 KB


None

,date_id,transaction_date,year,quarter,month,week,day_of_week,is_weekend
count,2282.000000,2282,2282.000000,2282.000000,2282.000000,2282.0,2282.000000,2282.000000
mean,1141.500000,2022-02-14 12:00:00,2021.638475,2.448729,6.343558,25.809378,3.000000,0.285714
min,1.000000,2019-01-01 00:00:00,2019.000000,1.000000,1.000000,1.0,0.000000,0.000000
25%,571.250000,2020-07-24 06:00:00,2020.000000,1.000000,3.000000,12.0,1.000000,0.000000
50%,1141.500000,2022-02-14 12:00:00,2022.000000,2.000000,6.000000,25.5,3.000000,0.000000
75%,1711.750000,2023-09-07 18:00:00,2023.000000,3.000000,9.000000,39.0,5.000000,1.000000
max,2282.000000,2025-03-31 00:00:00,2025.000000,4.000000,12.000000,53.0,6.000000,1.000000
std,658.900979,NaN,1.807814,1.133852,3.497644,15.259786,2.000438,0.451853


### Exchanges dimension

In [14]:
# Generate dim_exchange table with 4 exchanges (2 real, 2 fake)
EXCHANGES = [
    ('NYSE', 'USA', 'EST', 'USD'),
    ('NASDAQ', 'USA', 'EST', 'USD'),
    ('Global Digital Exchange', 'UK', 'GMT', 'GBP'),
    ('FutureTech Exchange', 'Singapore', 'SGT', 'SGD')
]

dim_exchange = pd.DataFrame({
    'exchange_id': range(1, len(EXCHANGES) + 1),
    'exchange_name': [e[0] for e in EXCHANGES],
    'country': [e[1] for e in EXCHANGES],
    'timezone': [e[2] for e in EXCHANGES],
    'currency': [e[3] for e in EXCHANGES]
})

In [15]:
df_overview(dim_exchange)

,exchange_id,exchange_name,country,timezone,currency
0,1,NYSE,USA,EST,USD
1,2,NASDAQ,USA,EST,USD
2,3,Global Digital Exchange,UK,GMT,GBP
3,4,FutureTech Exchange,Singapore,SGT,SGD


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   exchange_id    4 non-null      int64 
 1   exchange_name  4 non-null      object
 2   country        4 non-null      object
 3   timezone       4 non-null      object
 4   currency       4 non-null      object
dtypes: int64(1), object(4)
memory usage: 292.0+ bytes


None

,exchange_id
count,4.000000
mean,2.500000
std,1.290994
min,1.000000
25%,1.750000
50%,2.500000
75%,3.250000
max,4.000000


### Accounts dimensions

In [16]:
dim_account = pd.DataFrame({
    'account_id': range(1, NUM_ACCOUNTS + 1),
    'account_type': random.choices(['Retail', 'Institutional', 'Margin'], k=NUM_ACCOUNTS),
    'status': random.choices(['Active', 'Suspended', 'Closed'], k=NUM_ACCOUNTS),
    'opening_date': [fake.date_between(start_date='-5y', end_date='today') for _ in range(NUM_ACCOUNTS)],
    'risk_level': random.choices(['Low', 'Medium', 'High'], k=NUM_ACCOUNTS),
    'balance': [round(random.uniform(1000, 1000000), 2) for _ in range(NUM_ACCOUNTS)],
    'margin_enabled': random.choices([True, False], k=NUM_ACCOUNTS),
    'trading_experience': random.choices(['Beginner', 'Intermediate', 'Expert'], k=NUM_ACCOUNTS)
})

In [17]:
dim_account['margin_enabled'] = dim_account['margin_enabled'].map({True: 1, False: 0})

In [18]:
df_overview(dim_account)

,account_id,account_type,status,opening_date,risk_level,balance,margin_enabled,trading_experience
0,1,Margin,Active,2024-07-09,Medium,485566.23,1,Beginner
1,2,Retail,Closed,2023-05-06,Low,353539.14,1,Expert
2,3,Retail,Active,2023-03-27,High,133422.66,1,Intermediate
3,4,Margin,Active,2024-05-21,Medium,349185.89,1,Beginner
4,5,Institutional,Active,2022-09-19,High,970433.96,1,Expert


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246301 entries, 0 to 246300
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   account_id          246301 non-null  int64  
 1   account_type        246301 non-null  object 
 2   status              246301 non-null  object 
 3   opening_date        246301 non-null  object 
 4   risk_level          246301 non-null  object 
 5   balance             246301 non-null  float64
 6   margin_enabled      246301 non-null  int64  
 7   trading_experience  246301 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 15.0+ MB


None

,account_id,balance,margin_enabled
count,246301.000000,246301.000000,246301.000000
mean,123151.000000,500499.436306,0.499430
std,71101.118663,288129.090973,0.500001
min,1.000000,1000.500000,0.000000
25%,61576.000000,251729.770000,0.000000
50%,123151.000000,501188.710000,0.000000
75%,184726.000000,749627.700000,1.000000
max,246301.000000,999998.930000,1.000000


In [19]:
dim_account.to_csv("accounts_info.csv", index=False)

### Transactions table

In [19]:
# Generate fact_transactions
fact_transactions = pd.DataFrame({
    'transaction_id': range(1, NUM_TRANSACTIONS + 1),
    'account_id': random.choices(dim_account['account_id'], k=NUM_TRANSACTIONS),
    'stock_id': random.choices(dim_stock['stock_id'], k=NUM_TRANSACTIONS),
    'date_id': random.choices(dim_date['date_id'], k=NUM_TRANSACTIONS),
    'exchange_id': random.choices(dim_exchange['exchange_id'], k=NUM_TRANSACTIONS),
    'order_type': random.choices(['Buy', 'Sell'], k=NUM_TRANSACTIONS),
    'quantity': [random.randint(1, 1000) for _ in range(NUM_TRANSACTIONS)],
    'price': [round(random.uniform(50, 5000), 2) for _ in range(NUM_TRANSACTIONS)],  # Simulated price
})

# Calculate total value
fact_transactions['total_value'] = (fact_transactions['quantity'] * fact_transactions['price']).round(2)

/opt/homebrew/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/random.py:471: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return [population[floor(random() * n)] for i in _repeat(None, k)]


In [20]:
df_overview(fact_transactions)

,transaction_id,account_id,stock_id,date_id,exchange_id,order_type,quantity,price,total_value
0,1,244583,13,358,2,Buy,405,1919.91,777563.55
1,2,34657,98,1496,3,Buy,746,536.50,400229.00
2,3,53095,61,139,4,Buy,685,2741.36,1877831.60
3,4,145621,100,957,1,Sell,924,4661.20,4306948.80
4,5,171953,21,1260,3,Buy,467,3585.97,1674647.99


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   transaction_id  1200000 non-null  int64  
 1   account_id      1200000 non-null  int64  
 2   stock_id        1200000 non-null  int64  
 3   date_id         1200000 non-null  int64  
 4   exchange_id     1200000 non-null  int64  
 5   order_type      1200000 non-null  object 
 6   quantity        1200000 non-null  int64  
 7   price           1200000 non-null  float64
 8   total_value     1200000 non-null  float64
dtypes: float64(2), int64(6), object(1)
memory usage: 82.4+ MB


None

,transaction_id,account_id,stock_id,date_id,exchange_id,quantity,price,total_value
count,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06
mean,6.000005e+05,1.230865e+05,5.247688e+01,1.141846e+03,2.500114e+00,5.004701e+02,2.527942e+03,1.265075e+06
std,3.464103e+05,7.115345e+04,3.000408e+01,6.587891e+02,1.118213e+00,2.886576e+02,1.428278e+03,1.101838e+06
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.001000e+01,5.720000e+01
25%,3.000008e+05,6.144800e+04,2.700000e+01,5.710000e+02,1.000000e+00,2.500000e+02,1.291110e+03,3.537224e+05
50%,6.000005e+05,1.230940e+05,5.200000e+01,1.142000e+03,3.000000e+00,5.000000e+02,2.529190e+03,9.508456e+05
75%,9.000002e+05,1.847432e+05,7.800000e+01,1.712000e+03,4.000000e+00,7.510000e+02,3.764240e+03,1.929987e+06
max,1.200000e+06,2.463010e+05,1.040000e+02,2.282000e+03,4.000000e+00,1.000000e+03,4.999990e+03,4.993040e+06


## Save all generated data to local directory

In [21]:
generated_dfs_dict = {
    'dim_date': dim_date,
    'dim_account': dim_account,
    'dim_exchange': dim_exchange,
    'dim_stock': dim_stock,
    'fact_transactions': fact_transactions,
}

In [22]:
for file_name, df in generated_dfs_dict.items():
    df.to_csv(f'{dir_nz}/{file_name}.csv', index=False, sep="|")

## Data upload into Netezza

### Connection

In [23]:
# Define connection parameters
hostname = os.getenv('hostname')
port = int(os.getenv('port'))
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')
schema_name = os.getenv('schema')

# Connection string (no need for ODBC)
connection_string = f'jdbc:netezza://{hostname}:{port}/{database}'

In [24]:
# Create a connection to Netezza
try:
    conn = nzpy.connect(
        host=hostname,
        port=port,
        user=username,
        password=password,
        database=database
    )
    print("Connected to Netezza!")

    # Create a cursor to execute queries
    cursor = conn.cursor()

    # Example: List all tables in the database
    cursor.execute("SELECT schema, tablename FROM _v_table;")
    rows = cursor.fetchall()

    print("List of tables in the database:")
    for row in rows:
        print(f'{row[0]}.{row[1]}')

except Exception as e:
    print(f"Error: {e}")

Connected to Netezza!
List of tables in the database:
DEFINITION_SCHEMA._T_OBJECT
DEFINITION_SCHEMA._T_OBJECT_CLASSES
DEFINITION_SCHEMA._T_PROC
DEFINITION_SCHEMA._T_TYPE
DEFINITION_SCHEMA._T_ATTRIBUTE
DEFINITION_SCHEMA._T_CLASS
DEFINITION_SCHEMA._T_RELCHECK
DEFINITION_SCHEMA._T_ATTRDEF
DEFINITION_SCHEMA._T_TRIGGER
DEFINITION_SCHEMA._T_INHERITS
DEFINITION_SCHEMA._T_INDEX
DEFINITION_SCHEMA._T_STATISTIC
DEFINITION_SCHEMA._T_OPERATOR
DEFINITION_SCHEMA._T_OPCLASS
DEFINITION_SCHEMA._T_AM
DEFINITION_SCHEMA._T_AMOP
DEFINITION_SCHEMA._T_AMPROC
DEFINITION_SCHEMA._T_LANGUAGE
DEFINITION_SCHEMA._T_LARGEOBJECT
DEFINITION_SCHEMA._T_AGGREGATE
DEFINITION_SCHEMA._T_IPL
DEFINITION_SCHEMA._T_INHERITPROC
DEFINITION_SCHEMA._T_REWRITE
DEFINITION_SCHEMA._T_VIEW_USERSQL
DEFINITION_SCHEMA._T_LISTENER
DEFINITION_SCHEMA._T_DESCRIPTION
DEFINITION_SCHEMA._T_DIST_MAP
DEFINITION_SCHEMA._T_ORGANIZE_ON
DEFINITION_SCHEMA._T_GROOM_HISTORY
DEFINITION_SCHEMA._T_THIN
DEFINITION_SCHEMA._T_ALTBASE
DEFINITION_SCHEMA._T_SELCACH

### Create tables based on DDL

In [25]:
tables_ddl = {
'dim_account':
f'''
DROP TABLE {schema_name}.dim_account IF EXISTS ;
CREATE TABLE {schema_name}.dim_account (
    account_id INT PRIMARY KEY,  
    account_type VARCHAR(50),
    status VARCHAR(20),
    opening_date DATE,
    risk_level VARCHAR(10),
    balance DECIMAL(18,2),
    margin_enabled BOOLEAN,
    trading_experience VARCHAR(20)
);
''',
'dim_stock':
f'''
-- Stock Dimension
DROP TABLE {schema_name}.dim_stock IF EXISTS;
CREATE TABLE {schema_name}.dim_stock (
    stock_id INT PRIMARY KEY,
    stock_symbol VARCHAR(10) UNIQUE,                  
    stock_name VARCHAR(255),
    sector VARCHAR(100),
    industry VARCHAR(100),
    market_cap DECIMAL(18,2)  
);
''',
'dim_exchange':
f'''
-- Exchange Dimension
DROP TABLE {schema_name}.dim_exchange IF EXISTS;
CREATE TABLE {schema_name}.dim_exchange (
    exchange_id INT PRIMARY KEY,
    exchange_name VARCHAR(100) UNIQUE,
    country VARCHAR(50),
    timezone VARCHAR(10),
    currency VARCHAR(10)
);
''',
'dim_date':
f'''
-- Time Dimension
DROP TABLE {schema_name}.dim_date IF EXISTS;
CREATE TABLE {schema_name}.dim_date (
    date_id INT PRIMARY KEY,
    transaction_date DATE,
    year INT,
    quarter INT,
    month INT,
    week INT,
    day_of_week INT,
    is_weekend BOOLEAN
);
''',
'fact_transactions':
f'''
-- Fact Transactions
DROP TABLE {schema_name}.fact_transactions IF EXISTS;
CREATE TABLE {schema_name}.fact_transactions (
    transaction_id INT PRIMARY KEY,
    account_id INT REFERENCES dim_account(account_id),
    stock_id INT REFERENCES dim_stock(stock_id),
    date_id INT REFERENCES dim_date(date_id),
    exchange_id INT REFERENCES dim_exchange(exchange_id),
    order_type VARCHAR(10) CHECK (transaction_type IN ("BUY", "SELL")),
    quantity INT CHECK (shares > 0),
    price DECIMAL(10,2) CHECK (price > 0),
    total_value DECIMAL(18,2)
);'''
}

In [26]:
for t_n, ddl in tables_ddl.items():
    print(f'Creating {schema_name}.{t_n}')
    try:
        cursor.execute(ddl)
        print(f'Created {schema_name}.{t_n}')
    except Exception as e:
        print(f"Error creating table: {e}")

Creating EQUITY_TRANSACTIONS.dim_account
Created EQUITY_TRANSACTIONS.dim_account
Creating EQUITY_TRANSACTIONS.dim_stock
Created EQUITY_TRANSACTIONS.dim_stock
Creating EQUITY_TRANSACTIONS.dim_exchange
Created EQUITY_TRANSACTIONS.dim_exchange
Creating EQUITY_TRANSACTIONS.dim_date
Created EQUITY_TRANSACTIONS.dim_date
Creating EQUITY_TRANSACTIONS.fact_transactions
Created EQUITY_TRANSACTIONS.fact_transactions


### Insert data from local files

In [27]:
 # load data from local folder
for table_name in generated_dfs_dict:
    print(f"Processing {table_name}")
    # load data from local folder
    cursor.execute(f'''
    insert into {schema_name}.{table_name} 
        select * from external '{dir_nz}/{table_name}.csv'
        using (
            LogDir '{dir_logs}'
            remotesource 'python'
            dateDelim '-'
            delim '|'
            skiprows 1
        )
                ''')
    print(f"{cursor.rowcount} Rows loaded into the {table_name.upper()}")

Processing dim_date
2282 Rows loaded into the DIM_DATE
Processing dim_account
246301 Rows loaded into the DIM_ACCOUNT
Processing dim_exchange
4 Rows loaded into the DIM_EXCHANGE
Processing dim_stock
104 Rows loaded into the DIM_STOCK
Processing fact_transactions
1200000 Rows loaded into the FACT_TRANSACTIONS


## Close connection and clear logs

In [28]:
# Close the connection
if conn:
    conn.close()

### Clear logs

In [29]:
import shutil

def clear_folder(folder_path):
    # Loop through all files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        try:
            if os.path.isfile(file_path):  # Check if it's a file
                os.remove(file_path)  # Remove file
            elif os.path.isdir(file_path):  # If it's a directory, you can also remove it recursively
                shutil.rmtree(file_path)  # Remove the directory and its contents
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

# Example usage
folder_path = "./logs"
clear_folder(folder_path)